In [ ]:
import subprocess
from subprocess import Popen, PIPE
import time
import os
import sys
import argparse


Functions

In [ ]:
def run_cmd(mycmd):
    print("Running" + mycmd)
    cmd = mycmd
    out = subprocess.check_output(cmd, shell=True)
    print(out)

In [ ]:
def magic_run(cmd):
    !$cmd

In [ ]:
def delete_all_pods():
    cmd = "kubectl delete pod --all"
    !$cmd

In [ ]:
def delete_all_jobs():
    cmd = "kubectl delete job --all"
    !$cmd

In [ ]:
def launch_yaml(yamlfile):
    cmd = "kubectl create -f %s" % (yamlfile)
    !$cmd

function calls

In [ ]:
run_cmd("kubectl get pod")

In [117]:
magic_run("kubectl get job")

No resources found in ucr-rc namespace.


In [119]:
delete_all_jobs()

job.batch "cent" deleted


In [ ]:
delete_all_pods()

Misc

In [ ]:
# Get input from user
a = input()
print(a)

In [ ]:
k8scmdout = !kubectl get pod
print(k8scmdout)

In [ ]:
for x in range(6):
    magic_run("kubectl get pod")


In [ ]:
namedefault = "cent"
imagedefault = "centos:latest"
cmddefault = "uptime"


cmd = "./int.sh %s %s %s" % (namedefault, cmddefault, imagedefault)

!$cmd

In [ ]:
yaml_file = input()
launch_yaml(yaml_file)

In [123]:
job_name = "cent"
container_command = "ls -latr"
container_name = "thejobscontainer"
container_image = "centos:latest"
container__mem_limit = 4
container__cpu_limit = 4
container__mem_request = 1 
container__cpu_request = 1


cmd = """cat <<EOF | kubectl apply -f -
apiVersion: batch/v1
kind: Job
metadata:
  name: %s
spec:
  template:
    spec:
      containers:
      - name: %s
        image: %s
        command:
        - sh
        - -c
        - "%s"
        volumeMounts:
        - mountPath: /sharedvol
          name: sharedvol
        resources:
          limits:
            memory: %dGi
            cpu: "%d"
          requests:
            memory: %dGi
            cpu: "%d"
      volumes:
      - name: sharedvol
        persistentVolumeClaim:
          claimName: master-vol
      restartPolicy: Never
  backoffLimit: 1
EOF""" % (job_name,
          container_name,
          container_image,
          container_command,
          container__mem_limit,
          container__cpu_limit,
          container__mem_request,
          container__cpu_request)
!$cmd

podnamecmd = "kubectl get pods --no-headers --selector=job-name=%s | awk '{print $1}'" % (job_name)
podname = !$podnamecmd
podname = podname[0]
print(podname)

podlogcmd = "kubectl logs %s" % (podname)
!$podlogcmd
delete_all_jobs()

job.batch/cent created
cent-48slt
Error from server (BadRequest): container "thejobscontainer" in pod "cent-48slt" is waiting to start: ContainerCreating
job.batch "cent" deleted
